In [1]:
from pyspark.sql import SparkSession

In [29]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [3]:
spark = SparkSession.builder.master("local[5]").appName("BookSpark").getOrCreate()

25/07/06 07:47:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/06 07:47:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/06 07:47:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/07/06 07:47:01 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/07/06 07:47:01 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [5]:
book_ratings = spark.read.csv("certification/books/BX-Book-Ratings.csv", sep=";", header=True, inferSchema=True)

In [6]:
book_ratings.show(5)

+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
| 276727|0446520802|          0|
| 276729|052165615X|          3|
| 276729|0521795028|          6|
+-------+----------+-----------+
only showing top 5 rows



In [7]:
books = spark.read.csv("certification/books/BX-Books.csv", sep=";", header=True, inferSchema=True)

In [8]:
books.show(5)

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari

In [33]:
books = books.persist(StorageLevel.MEMORY_ONLY)

25/07/06 08:15:45 WARN CacheManager: Asked to cache already cached data.


In [42]:
books = books.repartition(1000, "Year-of-Publication")

In [43]:
users = spark.read.csv("certification/books/BX-Users.csv", sep=";", header=True, inferSchema=True)

In [44]:
users.show(5)

+-------+--------------------+----+
|User-ID|            Location| Age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|NULL|
|      2|stockton, califor...|  18|
|      3|moscow, yukon ter...|NULL|
|      4|porto, v.n.gaia, ...|  17|
|      5|farnborough, hant...|NULL|
+-------+--------------------+----+
only showing top 5 rows



### A. Find out the frequency of books published each year. (Hint: Use Boooks.csv file for this)

In [45]:
books.groupBy("Year-of-Publication").count().show()

+-------------------+-----+
|Year-of-Publication|count|
+-------------------+-----+
|               2003|14359|
|               1955|   70|
|               1978| 2131|
|               1925|    2|
|               1944|    4|
|               2038|    1|
|               1997|14892|
|               1806|    1|
|               1901|    7|
|               1971|  540|
|               1957|   76|
|               1945|    8|
|               1989| 7937|
|               1910|    1|
|               1965|  173|
|               1985| 5343|
|               1921|    2|
|               1987| 6529|
|               1941|   10|
|               1897|    1|
+-------------------+-----+
only showing top 20 rows



### B.Find out in which year maximum number of books were published

In [46]:
books.groupBy("Year-of-Publication").agg(count("Year-of-Publication").alias("Book Count")).orderBy(col("Book Count").desc()).show(1)

+-------------------+----------+
|Year-of-Publication|Book Count|
+-------------------+----------+
|               2002|     17628|
+-------------------+----------+
only showing top 1 row



### C. Find out how many book were published based on ranking in the year 2002. ( Hint: Use Book.csv and Book-Ratings.csv)

In [49]:
book_book_rating = books.join(book_ratings, on="ISBN", how="inner").where(books["Year-of-Publication"]==2002)

In [50]:
book_book_rating.groupBy(col("Book-Rating")).agg(count("Book-Rating").alias("Book Count")).orderBy(col("Book Count").desc()).show()

+-----------+----------+
|Book-Rating|Book Count|
+-----------+----------+
|          0|     53815|
|          8|      9881|
|          7|      6670|
|          9|      6564|
|         10|      6273|
|          5|      3614|
|          6|      3186|
|          4|       861|
|          3|       534|
|          2|       260|
|          1|       143|
+-----------+----------+

